In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Features.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4993 entries, 0 to 4992
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           4993 non-null   int64  
 1   Unnamed: 0             4993 non-null   int64  
 2   name_length            4993 non-null   float64
 3   is_organic             4993 non-null   float64
 4   quantity_value         4993 non-null   float64
 5   quantity_unit_encoded  4993 non-null   float64
 6   category_depth         4993 non-null   float64
 7   country_count          4993 non-null   float64
 8   product_age_days       4993 non-null   float64
 9   created_month          4993 non-null   float64
 10  main_category_encoded  4993 non-null   float64
 11  nova_group             4993 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 468.2 KB


# OVERFITTING

# max_depth=None + DecisionTreeClassifier

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# === 1️⃣ Ma'lumotlarni tayyorlash ===
# Masalan, maqsad ustuni 'nova_group' bo'lsin (o'zingiznikini yozing)
X = df.drop(columns=['nova_group'], errors='ignore')
y = df['nova_group']

# === 2️⃣ Raqamli va kategorik ustunlarni aniqlash ===
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# === 3️⃣ Oldindan ishlov berish (scaling + one-hot encoding) ===
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# === 4️⃣ Model pipeline ===
model = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=None, random_state=42))
])

# === 5️⃣ Train/Test bo‘lish ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# === 6️⃣ Modelni o‘qitish ===
model.fit(X_train, y_train)

# === 7️⃣ Baholash ===
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("🔥 CLASSIFICATION MODEL RESULTS")
print("-------------------------------")
print(f"✅ Train Accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
print(f"✅ Test  Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
print("\n🧾 Classification Report:")
print(classification_report(y_test, y_test_pred))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

🔥 CLASSIFICATION MODEL RESULTS
-------------------------------
✅ Train Accuracy: 1.000
✅ Test  Accuracy: 0.439

🧾 Classification Report:
              precision    recall  f1-score   support

         1.0       0.23      0.24      0.23       170
         2.0       0.16      0.19      0.17        64
         3.0       0.30      0.29      0.29       330
         4.0       0.59      0.58      0.58       787
         5.0       0.34      0.37      0.35       147

    accuracy                           0.44      1498
   macro avg       0.32      0.33      0.33      1498
weighted avg       0.44      0.44      0.44      1498


📊 Confusion Matrix:
[[ 40   9  26  82  13]
 [  9  12  11  25   7]
 [ 29  22  95 160  24]
 [ 76  24 170 457  60]
 [ 17   8  16  52  54]]


# ❌ Overfitting

In [7]:
# === 8️⃣ Modelni saqlash ===
import os
save_dir = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models"
os.makedirs(save_dir, exist_ok=True)  # papkani avtomatik yaratadi, agar mavjud bo'lmasa

model_path = os.path.join(save_dir, "DecisionTree_nova_group_pipeline.joblib")
joblib.dump(model, model_path)

print(f"\n✅ Model muvaffaqiyatli saqlandi:\n{model_path}")


✅ Model muvaffaqiyatli saqlandi:
C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecisionTree_nova_group_pipeline.joblib


In [9]:
import joblib

# Fayl yo‘li
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecisionTree_nova_group_pipeline.joblib"

# Modelni yuklash
loaded_model = joblib.load(model_path)

print("✅ Model muvaffaqiyatli yuklandi!")

✅ Model muvaffaqiyatli yuklandi!


In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib

# --- 1️⃣ Modelni yuklash ---
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecisionTree_nova_group_pipeline.joblib"

loaded_model = joblib.load(model_path)

# --- 2️⃣ Yangi ma'lumot (unseen data) ---
new_row = pd.DataFrame({
    'Unnamed: 0.1': [999],
    'Unnamed: 0': [999],
    'name_length': [3],
    'is_organic': [1],
    'quantity_value': [100.0],
    'quantity_unit_encoded': [7],
    'category_depth': [5],
    'country_count': [2],
    'product_age_days': [2500],
    'created_month': [6],
    'main_category_encoded': [159]    
})

# --- 3️⃣ Yangi ma'lumot uchun bashorat ---
pred_target = loaded_model.predict(new_row)[0]

# --- 4️⃣ Test RMSE hisoblash (agar y_test va y_test_pred mavjud bo‘lsa) ---
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# --- 5️⃣ Natijani chiqarish ---
print("\n--- Offline (Unseen) nova_group Prediction ---")
print(f"🧠 Bashorat qilingan nova_group qiymati: {pred_target:.3f}")
print(f"📏 Model xatoligi (RMSE test setdan): ±{rmse_test:.3f}")


--- Offline (Unseen) nova_group Prediction ---
🧠 Bashorat qilingan nova_group qiymati: 4.000
📏 Model xatoligi (RMSE test setdan): ±1.196


# RMSE = 1.196 bo‘lsa, bu degani model har bir obyekt uchun o‘rtacha 1.196 daraja farq bilan noto‘g‘ri bashorat qilmoqda.
#Ya’ni, haqiqiy nova_group = 4 bo‘lsa, model ko‘pincha ≈3 yoki ≈2.8 deb aytadi.

# UNDERFITTING

# max_depth=1  + DecisionTreeClassifier

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# --- 1️⃣ Ma'lumotlar tayyorlash ---
# Masalan, maqsad ustuni 'nova_group' bo‘lsin
X = df.drop(columns=['nova_group'], errors='ignore')
y = df['nova_group']

# --- 2️⃣ Raqamli va kategorik ustunlarni aniqlash ---
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# --- 3️⃣ Oldindan ishlov berish (scaling + one-hot encoding) ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# --- 4️⃣ Underfitting model ---
underfit_model1 = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=1, random_state=42))  # juda sayoz daraxt
])

# --- 5️⃣ Train/Test bo‘lish ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# --- 6️⃣ Modelni o‘qitish ---
underfit_model1.fit(X_train, y_train)

# --- 7️⃣ Baholash ---
y_train_pred = underfit_model1.predict(X_train)
y_test_pred = underfit_model1.predict(X_test)

print("🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS")
print("--------------------------------------------")
print(f"✅ Train Accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
print(f"✅ Test  Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
print("\n🧾 Classification Report:")
print(classification_report(y_test, y_test_pred))
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS
--------------------------------------------
✅ Train Accuracy: 0.526
✅ Test  Accuracy: 0.525

🧾 Classification Report:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       170
         2.0       0.00      0.00      0.00        64
         3.0       0.00      0.00      0.00       330
         4.0       0.53      1.00      0.69       787
         5.0       0.00      0.00      0.00       147

    accuracy                           0.53      1498
   macro avg       0.11      0.20      0.14      1498
weighted avg       0.28      0.53      0.36      1498


📊 Confusion Matrix:
[[  0   0   0 170   0]
 [  0   0   0  64   0]
 [  0   0   0 330   0]
 [  0   0   0 787   0]
 [  0   0   0 147   0]]


c:\Users\Rasulbek907\Desktop\Project_MP\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rasulbek907\Desktop\Project_MP\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capi

# ❌ Underfitting

# max_depth=20 + DecisionTreeClassifier

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# --- 1️⃣ Ma'lumotlar tayyorlash ---
# Masalan, maqsad ustuni 'nova_group' bo‘lsin
X = df.drop(columns=['nova_group'], errors='ignore')
y = df['nova_group']

# --- 2️⃣ Raqamli va kategorik ustunlarni aniqlash ---
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# --- 3️⃣ Oldindan ishlov berish (scaling + one-hot encoding) ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# --- 4️⃣ Underfitting model ---
underfit_model = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=20, random_state=42))  # juda sayoz daraxt
])

# --- 5️⃣ Train/Test bo‘lish ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# --- 6️⃣ Modelni o‘qitish ---
underfit_model.fit(X_train, y_train)

# --- 7️⃣ Baholash ---
y_train_pred = underfit_model.predict(X_train)
y_test_pred = underfit_model.predict(X_test)

print("🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS")
print("--------------------------------------------")
print(f"✅ Train Accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
print(f"✅ Test  Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
print("\n🧾 Classification Report:")
print(classification_report(y_test, y_test_pred))
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS
--------------------------------------------
✅ Train Accuracy: 0.975
✅ Test  Accuracy: 0.452

🧾 Classification Report:
              precision    recall  f1-score   support

         1.0       0.25      0.25      0.25       170
         2.0       0.22      0.22      0.22        64
         3.0       0.33      0.31      0.32       330
         4.0       0.59      0.59      0.59       787
         5.0       0.32      0.37      0.34       147

    accuracy                           0.45      1498
   macro avg       0.34      0.35      0.34      1498
weighted avg       0.45      0.45      0.45      1498


📊 Confusion Matrix:
[[ 43   6  23  83  15]
 [  7  14  13  25   5]
 [ 33  19 103 156  19]
 [ 73  18 159 463  74]
 [ 16   7  15  55  54]]


# max_depth=8, min_samples_split=10, min_samples_leaf=5 +  DecisionTreeClassifier

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# --- 1️⃣ Ma'lumotlar tayyorlash ---
# Masalan, maqsad ustuni 'nova_group' bo‘lsin
X = df.drop(columns=['nova_group'], errors='ignore')
y = df['nova_group']

# --- 2️⃣ Raqamli va kategorik ustunlarni aniqlash ---
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# --- 3️⃣ Oldindan ishlov berish (scaling + one-hot encoding) ---
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# --- 4️⃣ Underfitting model ---
underfit_model = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=5))
])

# --- 5️⃣ Train/Test bo‘lish ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# --- 6️⃣ Modelni o‘qitish ---
underfit_model.fit(X_train, y_train)

# --- 7️⃣ Baholash ---
y_train_pred = underfit_model.predict(X_train)
y_test_pred = underfit_model.predict(X_test)

print("🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS")
print("--------------------------------------------")
print(f"✅ Train Accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
print(f"✅ Test  Accuracy: {accuracy_score(y_test, y_test_pred):.3f}")
print("\n🧾 Classification Report:")
print(classification_report(y_test, y_test_pred))
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


🔥 UNDERFITTING CLASSIFICATION MODEL RESULTS
--------------------------------------------
✅ Train Accuracy: 0.613
✅ Test  Accuracy: 0.525

🧾 Classification Report:
              precision    recall  f1-score   support

         1.0       0.31      0.07      0.11       170
         2.0       0.30      0.05      0.08        64
         3.0       0.32      0.21      0.25       330
         4.0       0.58      0.84      0.68       787
         5.0       0.47      0.29      0.36       147

    accuracy                           0.53      1498
   macro avg       0.40      0.29      0.30      1498
weighted avg       0.47      0.53      0.47      1498


📊 Confusion Matrix:
[[ 12   0  24 122  12]
 [  4   3  14  40   3]
 [  6   3  68 245   8]
 [ 12   2  87 662  24]
 [  5   2  18  80  42]]


In [19]:
# === 8️⃣ Modelni saqlash ===
import os
save_dir = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models"
os.makedirs(save_dir, exist_ok=True)  # papkani avtomatik yaratadi, agar mavjud bo'lmasa

model_path = os.path.join(save_dir, "DecTre_undefitting.joblib")
joblib.dump(underfit_model1, model_path)

print(f"\n✅ Model muvaffaqiyatli saqlandi:\n{model_path}")


✅ Model muvaffaqiyatli saqlandi:
C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecTre_undefitting.joblib


In [20]:
import joblib

# Fayl yo‘li
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecTre_undefitting.joblib"

# Modelni yuklash
loaded_model = joblib.load(model_path)

print("✅ Model muvaffaqiyatli yuklandi!")

✅ Model muvaffaqiyatli yuklandi!


In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib

# --- 1️⃣ Modelni yuklash ---
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecisionTree_nova_group_pipeline.joblib"
loaded_model = joblib.load(model_path)

# --- 2️⃣ 5 ta yangi ma'lumot (unseen data) ---
new_rows = pd.DataFrame([
    {
        'Unnamed: 0.1': 2001, 'Unnamed: 0': 2001, 'name_length': 2,
        'is_organic': 1, 'quantity_value': 50.0, 'quantity_unit_encoded': 5,
        'category_depth': 3, 'country_count': 1, 'product_age_days': 3000,
        'created_month': 1, 'main_category_encoded': 100
    },
    {
        'Unnamed: 0.1': 2002, 'Unnamed: 0': 2002, 'name_length': 7,
        'is_organic': 0, 'quantity_value': 500.0, 'quantity_unit_encoded': 9,
        'category_depth': 10, 'country_count': 6, 'product_age_days': 1200,
        'created_month': 12, 'main_category_encoded': 275
    },
    {
        'Unnamed: 0.1': 2003, 'Unnamed: 0': 2003, 'name_length': 4,
        'is_organic': 0, 'quantity_value': 120.0, 'quantity_unit_encoded': 4,
        'category_depth': 6, 'country_count': 3, 'product_age_days': 2200,
        'created_month': 3, 'main_category_encoded': 150
    },
    {
        'Unnamed: 0.1': 2004, 'Unnamed: 0': 2004, 'name_length': 6,
        'is_organic': 1, 'quantity_value': 75.0, 'quantity_unit_encoded': 8,
        'category_depth': 4, 'country_count': 2, 'product_age_days': 900,
        'created_month': 8, 'main_category_encoded': 210
    },
    {
        'Unnamed: 0.1': 2005, 'Unnamed: 0': 2005, 'name_length': 3,
        'is_organic': 1, 'quantity_value': 300.0, 'quantity_unit_encoded': 6,
        'category_depth': 9, 'country_count': 5, 'product_age_days': 1500,
        'created_month': 10, 'main_category_encoded': 350
    }
])

# --- 3️⃣ 5 ta yangi ma'lumot uchun bashorat ---
predictions = loaded_model.predict(new_rows)

# --- 4️⃣ Agar y_test mavjud bo‘lsa (ixtiyoriy) ---
# (faqat sizda avvalgi kodda y_test va y_test_pred mavjud bo‘lsa ishlaydi)
try:
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
except NameError:
    rmse_test = None

# --- 5️⃣ Natijalarni chiqarish ---
print("\n--- 🧠 5 ta Offline (Unseen) nova_group Bashoratlari ---")
for i, (idx, row) in enumerate(new_rows.iterrows()):
    print(f"{i+1}) Mahsulot ID: {row['Unnamed: 0']}  |  Bashorat qilingan nova_group: {predictions[i]}")

if rmse_test is not None:
    print(f"\n📏 Model xatoligi (RMSE test setdan): ±{rmse_test:.3f}")



--- 🧠 5 ta Offline (Unseen) nova_group Bashoratlari ---
1) Mahsulot ID: 2001.0  |  Bashorat qilingan nova_group: 4.0
2) Mahsulot ID: 2002.0  |  Bashorat qilingan nova_group: 4.0
3) Mahsulot ID: 2003.0  |  Bashorat qilingan nova_group: 4.0
4) Mahsulot ID: 2004.0  |  Bashorat qilingan nova_group: 4.0
5) Mahsulot ID: 2005.0  |  Bashorat qilingan nova_group: 4.0

📏 Model xatoligi (RMSE test setdan): ±1.229


In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib

# --- 1️⃣ Modelni yuklash ---
model_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Models\Pipeline_Models\DecisionTree_nova_group_pipeline.joblib"
loaded_model = joblib.load(model_path)

# --- 2️⃣ 5 ta yangi ma'lumot (unseen data) ---
new_rows = pd.DataFrame([
    # 🥦 1. Kichik, organik, eski mahsulot
    {
        'Unnamed: 0.1': 3001, 'Unnamed: 0': 3001, 'name_length': 2,
        'is_organic': 1, 'quantity_value': 50.0, 'quantity_unit_encoded': 5,
        'category_depth': 3, 'country_count': 1, 'product_age_days': 4000,
        'created_month': 1, 'main_category_encoded': 50
    },
    # 🍫 2. Katta, no-organik, yangi mahsulot
    {
        'Unnamed: 0.1': 3002, 'Unnamed: 0': 3002, 'name_length': 8,
        'is_organic': 0, 'quantity_value': 1000.0, 'quantity_unit_encoded': 9,
        'category_depth': 10, 'country_count': 8, 'product_age_days': 100,
        'created_month': 12, 'main_category_encoded': 300
    },
    # 🧴 3. O‘rtacha, organik
    {
        'Unnamed: 0.1': 3003, 'Unnamed: 0': 3003, 'name_length': 4,
        'is_organic': 1, 'quantity_value': 250.0, 'quantity_unit_encoded': 7,
        'category_depth': 6, 'country_count': 3, 'product_age_days': 2000,
        'created_month': 6, 'main_category_encoded': 150
    },
    # 🥤 4. Kichik, yangi, ko‘p davlatda mavjud
    {
        'Unnamed: 0.1': 3004, 'Unnamed: 0': 3004, 'name_length': 5,
        'is_organic': 0, 'quantity_value': 75.0, 'quantity_unit_encoded': 6,
        'category_depth': 4, 'country_count': 5, 'product_age_days': 500,
        'created_month': 8, 'main_category_encoded': 220
    },
    # 🍎 5. Juda katta, eski, no-organik
    {
        'Unnamed: 0.1': 3005, 'Unnamed: 0': 3005, 'name_length': 10,
        'is_organic': 0, 'quantity_value': 2000.0, 'quantity_unit_encoded': 10,
        'category_depth': 12, 'country_count': 10, 'product_age_days': 3500,
        'created_month': 3, 'main_category_encoded': 400
    }
])

# --- 3️⃣ 5 ta yangi ma'lumot uchun bashorat ---
predictions = loaded_model.predict(new_rows)


# --- 4️⃣ Agar y_test mavjud bo‘lsa (ixtiyoriy) ---
# (faqat sizda avvalgi kodda y_test va y_test_pred mavjud bo‘lsa ishlaydi)
try:
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
except NameError:
    rmse_test = None

# --- 5️⃣ Natijalarni chiqarish ---
print("\n--- 🧠 5 ta Offline (Unseen) nova_group Bashoratlari ---")
for i, (idx, row) in enumerate(new_rows.iterrows()):
    print(f"{i+1}) Mahsulot ID: {row['Unnamed: 0']}  |  Bashorat qilingan nova_group: {predictions[i]}")

if rmse_test is not None:
    print(f"\n📏 Model xatoligi (RMSE test setdan): ±{rmse_test:.3f}")



--- 🧠 5 ta Offline (Unseen) nova_group Bashoratlari ---
1) Mahsulot ID: 3001.0  |  Bashorat qilingan nova_group: 4.0
2) Mahsulot ID: 3002.0  |  Bashorat qilingan nova_group: 4.0
3) Mahsulot ID: 3003.0  |  Bashorat qilingan nova_group: 4.0
4) Mahsulot ID: 3004.0  |  Bashorat qilingan nova_group: 4.0
5) Mahsulot ID: 3005.0  |  Bashorat qilingan nova_group: 4.0

📏 Model xatoligi (RMSE test setdan): ±1.229
